Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 
- Explore tabular data for supervised machine learning
- Join relational data for supervised machine learning

# Explore tabular data for superviesd machine learning 🍌

Wrangling your dataset is often the most challenging and time-consuming part of the modeling process.

In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

Let’s get set up:

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/home/alex/code/DS-Unit-2-Applied-Modeling/module2-wrangle-ml-datasets/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

order_products__train.csv
orders.csv
order_products__prior.csv
departments.csv
products.csv
aisles.csv


In [6]:
# For each csv file, look at its shape & head
import pandas as pd
from IPython.display import display

def preview():
    for filename in glob("*.csv"):
        df = pd.read_csv(filename)
        print("\n",filename, df.shape)
        display(df.head())

preview()



 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0



 order_products__prior.csv (32434489, 4)


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13



 aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


### The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

### ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

### Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## Follow Along

### What was the most frequently ordered product?

In [7]:
order_products_train = pd.read_csv("order_products__train.csv")

In [11]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [15]:
sorted_product_ids = order_products_train.groupby("product_id")["reordered"].sum().sort_values(ascending=False)

In [16]:
products = pd.read_csv("products.csv")
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [18]:
products_and_order_freq = products.merge(sorted_product_ids, on="product_id")

In [23]:
product_id_name_and_ordercount = products_and_order_freq.sort_values("reordered", ascending=False)
product_id_name_and_ordercount

,product_id,product_name,aisle_id,department_id,reordered
19574,24852,Banana,24,4,16557
10367,13176,Bag of Organic Bananas,24,4,13362
16685,21137,Organic Strawberries,24,4,8603
17300,21903,Organic Baby Spinach,123,4,8055
37609,47766,Organic Avocado,24,4,6226
...,...,...,...,...,...
20846,26461,Green Peppercorns,104,13,0
32519,41292,Chewable Tablets Esberitox,11,11,0
20848,26464,Premium Green Tea Decaf Tea Bags,94,7,0
11817,15047,Tea Light Candles,101,17,0


### How often are bananas included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include bananas?"

Let's wrangle!

In [25]:
products = pd.read_csv("products.csv")
train = pd.merge(order_products_train, products, on='product_id', how='inner')

In [28]:
train.shape


(1384617, 7)

In [29]:
train.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16


In [40]:
train["Is Banana"] = (train["product_name"] == "Banana") | (train["product_name"] == "Bag of Organic Bananas") | (train["product_name"] == "Organic Banana") | (train["product_name"] == "Bananas")

In [45]:
train["Is Banana"].value_counts(normalize=True)*100

False    97.335148
True      2.664852
Name: Is Banana, dtype: float64

In [56]:
train.groupby("reordered")["Is Banana"].value_counts(normalize=True)

reordered  Is Banana
0          False        0.990378
           True         0.009622
1          False        0.961934
           True         0.038066
Name: Is Banana, dtype: float64

# Join relational data for supervised machine learning

## Overview
Often, you’ll need to join data from multiple relational tables before you’re ready to fit your models.

### Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [57]:
orders = pd.read_csv("orders.csv")

In [58]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [60]:
train_and_orders = train.merge(orders, on="order_id")
train_and_orders.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,Is Banana,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,False,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,False,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,4,False,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,4,False,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,False,112108,train,4,4,10,9.0


In [87]:
banana_eaters = train_and_orders[train_and_orders["Is Banana"] == True]["user_id"].drop_duplicates().tolist()

## Follow Along

### How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought bananas.

(And *none* of the orders from customers who have *never* bought bananas.)

In [89]:
train_and_orders[train_and_orders["user_id"].isin(banana_eaters)]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,Is Banana,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,False,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,False,112108,train,4,4,10,9.0
2,1,10246,3,0,Organic Celery Hearts,83,4,False,112108,train,4,4,10,9.0
3,1,49683,4,0,Cucumber Kirby,83,4,False,112108,train,4,4,10,9.0
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,False,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372879,1022064,34320,3,0,Organic Apples Pommes,24,4,False,196569,train,4,5,14,15.0
1372880,1022064,13798,1,0,French Roast Organic Whole Bean Coffee,26,7,False,196569,train,4,5,14,15.0
1372881,1022064,9583,2,0,Brass Tacks Blend Whole Bean Coffee,26,7,False,196569,train,4,5,14,15.0
1372882,1618153,39276,1,0,Bananas,24,4,True,26935,train,52,1,6,4.0


## What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

- Products per order
- Time of day
- Have they reordered bananas before? (Have ordered bananas >= 2 times)
- Other fruit they buy 
- Size of orders (customers with smaller orders on average are less likely to be reordering any particular product on their next order)


- Frequency of banana orders:
    - % of orders 
    - Time between banana orders: Every n days on average
    - Raw count: Total orders, how many times have you ordered bananas?

- Recency of banana orders
    - n days since you ordered banana



## Challenge

**Continue to clean and explore your data.** Can you **engineer features** to help predict your target? For the evaluation metric you chose, what score would you get just by guessing? Can you **make a fast, first model** that beats guessing?

We recommend that you use your portfolio project dataset for all assignments this sprint. But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset today. Follow the instructions in the assignment notebook. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!